## TENSORFLOW BASICS ##

<font color=green>
MA5203 Aprendizaje de Máquinas Probabilístico
<br>
Autor: Nicolás Aramayo
</font>

TensorFlow es una librería open source para computación numérica de alto desempeño, especialmente diseñado para Machine Learning de gran escala. Su arquitectura flexible permite el despliegue de computación a través de múltiples plataformas (CPUs, GPUs, TPUs), y desde estacionarios a clusters de servidores a dispositivos móviles. Fue originalmente desarrollado por investigadores e ingenieros de Google Brain dentro de la organización de AI de Google, y tiene un fuerte soporte para machine learning y deep learning.

https://www.tensorflow.org/

Para instalar TensorFlow, se deben cumplir los siguientes requerimientos:<br>
> -64-bit, x86 estacionarios o laptops
<br>
> -Windows 7 o más

Si Python fue instalado usando Anaconda, se recomienda abrir el Prompt Anaconda y ejecutar: pip install --upgrade tensorflow
<br>(Esto se hace para instalar TensorFlow en un virtual environment)

Más información: https://www.tensorflow.org/install/
<br>
***

In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

### Print 'Hello, TensorFlow!'

In [ ]:
hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

***

Un programa de TensorFlow típicamente se divide en dos partes: primero construye un grafo computacional (__fase de construcción__), y luego ejecútalo (__fase de ejecución__). Su forma de trabajar es simple, primero definiendo un grafo de computaciones en Python, y luego TensorFlow toma este grafo para ejecutarlo en C++.

-__Fase de construcción__: construye un grafo computacional representando al modelo de Machine Learning y las computaciones necesarias para entrenarlo.

-__Fase de ejecución__: ejecuta un loop que evalúa un training step repetidamente, gradualmente mejorando los parámetros del modelo.


### Crea un Grafo Computacional y Ejecútalo

In [ ]:
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

Primero, debes abrir una __sesión__ de TensorFlow. Las sesiones se hacen cargo de llevar las operaciones a dispositivos (CPUs, GPUs) y ejecutarlos

In [ ]:
sess = tf.Session()

#Inicializa las variables
sess.run(x.initializer)
sess.run(y.initializer)

#Evalúa
result = sess.run(f)
print(result)

#Cierra la sesión
sess.close()

La siguiente imagen muestra el grafo computacional recién implementado. Toma como inputs los nodos _x_ e _y_ y aplica las operaciones detalladas en _f_ para generar el output

<img src="https://github.com/nicoaramayo/imgsTFBasics/raw/master/grafo_1.PNG" alt="Drawing" style="width: 500px;"/>

Una forma más simple de ejecutar una sesión

In [ ]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    print(result)
#Esta sesión fue cerrada automáticamente

O mejor aún, inicializa automáticamente todas las variables creando un nodo 'init'

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

Como en el ejemplo, una __variable__ en TensorFlow se añade al grafo computacional al construir una instancia de la clase Variable. El constructor Variable() necesita un valor inicial para la variable:

                                w = tf.Variable(<valor-inicial>, name=<nombre-opcional>)
                         
Para cambiar el valor de la variable se puede usar el método __assign()__

### Manejando Grafos

Cualquier nodo creado se añade automáticamente al grafo default

In [ ]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

Para manejar múltiples grafos independientes, crea un nuevo __Grafo__ y temporalmente ejecútalo como el grafo default dentro de un bloque _with_

In [ ]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

En Jupyter, ejecutar una misma celda más de una vez puede resultar en que un grafo default contenga muchos nodos duplicados. Una solución es reiniciar el _kernel_, o mejor aún, reiniciar el grafo default:

In [ ]:
tf.reset_default_graph()

Al evaluar un nodo, TensorFlow automáticamente determinará el set de nodos de los cuales depende y evaluará estos nodos primero

In [ ]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

Todos los valores en los nodos se eliminan entre ejecuciones del grafo excepto los valores variables, los cuales son mantenidos por la sesión entre ejecuciones del grafo. Una variable comienza su vida cuando se ejecuta el inicializador, y termina cuando la sesión es cerrada

Para evaluar de manera eficiente _y_ y _z_, pídele a TensorFlow que los evalúe ambos en una misma ejecución del grafo:

In [ ]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

### Ejemplo 1: Implementación de Descenso del Gradiente en una Regresión Lineal

Importa el dataset California Housing, que contiene 8 atributos sobre casas (contenidos en _housing.data_) y la valoración de la casa como la variable dependiente (contenido en _housing.target_)

In [ ]:
from sklearn.datasets import fetch_california_housing

tf.reset_default_graph()

housing = fetch_california_housing()

#Obtiene las dimensiones de la matriz de covariables X
m, n = housing.data.shape

In [ ]:
housing.data

Al usar Descenso del Gradiente es importante normalizar los vectores de input feature, o de otra forma el entrenamiento podría ser mucho más lento. Piensa en descender por una tazón alargado versus descender por un tazón con una curvatura simétrica: el camino por cubrir podría ser mucho más corto!

Para realizar esto se usará la función _StandardScaler_ de _sklearn_

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
#Añade el bias
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [ ]:
#Define el número de iteraciones y el learning rate
n_epochs = 1000
learning_rate = 0.01

#Define X e y como constantes de TensorFlow, y theta como una variable de TensorFlow ya que será constantemente actualizada
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
#Obtén los valores iniciales de manera aleatoria, con las dimensiones correctas
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")

y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")

#Guardamos los valores del MSE en una lista
costs = []

#Usa el optimizador Gradient Descent implementado en TensorFlow, y entrena para optimizar el mean squared error
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
        costs.append(mse.eval())
    
    #Recupera el theta optimizado
    best_theta = theta.eval()
    print("MSE óptimo luego de " + str(n_epochs) + " iteraciones:")
    print(mse.eval())

print("Best theta:")
print(best_theta)

#Obtiene el óptimo analítico para comparación
X_ols = scaled_housing_data_plus_bias
y_ols = housing.target.reshape(-1, 1)
y_pred_ols = np.matmul(X_ols, np.linalg.lstsq(X_ols, y_ols)[0])
mse_ols = np.mean(np.square(y_pred_ols - y_ols))
mse_in_time = [mse_ols] * n_epochs

plt.plot(np.squeeze(costs), 'b')
plt.plot(mse_in_time, 'r')
plt.ylabel('Mean Squared Error')
plt.xlabel('Iterations')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

### Ejemplo 1.1: Implementa Descenso del Gradiente por Mini-Batches para una Regresión Lineal

Para Descenso del Gradiente por Mini-Batches, _X_ e _y_ deben ser reemplazados en cada iteración con el siguiente mini-batch. Esto se puede realizar usando nodos __placeholder__. Estos nodos no realizan ninguna operación, solo entregan como output la data que tú les pides que entreguen durante la ejecución. Típicamente son usados para entregar la data de entrenamiento a TensorFlow durante el entrenamiento. Para la evaluación, usa __feed_dict__ en el método _eval()_ durante el entrenamiento

In [ ]:
tf.reset_default_graph()

#Define X e y como nodos placeholder
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

In [ ]:
n_epochs = 100
learning_rate = 0.01

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
costs = []
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

#Elige el tamaño del batch para realizar Descenso del Gradiente por Mini-Batches,
#y la cantidad de batches será el número de filas sobre el tamaño de los batches
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    '''
    Función para obtener batches de X e y de tamaño batch_size
    '''
    np.random.seed(epoch * n_batches + batch_index)
    #Obtén una cantidad batch_size de índices de 0 a m-1
    indices = np.random.randint(m, size=batch_size)
    #Elige en X e y las filas de acuerdo con los índices sampleados
    X_batch = scaled_housing_data_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        
        epoch_cost = 0.
        
        for batch_index in range(n_batches):
            
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            #Para evaluar, entrega X_batch y y_batch usando feed_dict en la ejecución
            _, minibatch_cost = sess.run([training_op, mse], feed_dict={X: X_batch, y: y_batch})
            epoch_cost += minibatch_cost / n_batches
        costs.append(epoch_cost)

    best_theta = theta.eval()
    
print("Best theta:")
print(best_theta)

mse_in_time = [mse_ols] * n_epochs

plt.plot(np.squeeze(costs), 'b')
plt.plot(mse_in_time, 'r')
plt.ylabel('Mean Squared Error')
plt.xlabel('Iterations')
plt.title("Learning rate =" + str(learning_rate))
plt.show()

### Compara Resultados (Opcional)
Para comparar los resultados entre ambos algoritmos de optimización, ejecuta una sesión para obtener el MSE usando el 'best_theta' obtenido en el último bloque con Descenso del Gradiente por Mini-Batches. También puedes ajustar los hiperparámetros learning rate y/o el número de epochs para la comparación de resultados

In [ ]:
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
y_pred = tf.matmul(X, best_theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    print("MSE obtenido con Descenso del Gradiente por Mini-Batches:")
    print(mse.eval())

### Guarda y Restaura tu Modelo

Una vez entrenado tu modelo, podrías querer guardar los parámetros obtenidos para luego poder usarlos cuando lo requieras, para usarlos en otro programa, para compararlos con otros modelos, etc. Más aún, podrías querer guardar checkpoints en intervalos regulares de tiempo de manera que si tu computador falla puedas continuar desde el último checkpoit, o para realizar regularización por Early Stopping.

En TensorFlow crea un nodo __Saver__ al final de la fase de construcción (luego de que todos los nodos de variables han sido creados), y luego en la fase de ejecución solo llama a su método __save()__ cuando desees, entregándole la sesión y el _path_ del archivo checkpoint

In [ ]:
tf.reset_default_graph()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y") 
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

#Define el nodo saver
saver = tf.train.Saver([theta])
#Usa el siguiente Saver solo si quieres recuperar los valores de theta bajo el nombre "weigths"
#saver = tf.train.Saver({"weights": theta})

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            #Guarda el nodo saver cada 10 epochs, entregando la sesión y el directorio correspondiente
            #con el archivo checkpoint de nombre 'my_model' que será tipo .ckpt
            save_path = saver.save(sess, "./tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    #Guarda el resultado luego de completar el entrenamiento, llamando al archivo ahora 'my_model_final'
    save_path = saver.save(sess, "./tmp/my_model_final.ckpt")

Para restaurar un modelo, al comienzo de la fase de ejecución en vez de inicializar las variables usando el nodo _init_, llama al método __restore()__ del objeto _Saver_

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()
    
print('Best theta restored:')
print(best_theta_restored)

Por default, el método __save()__ también guarda la estructura del grafo en un segundo archivo del mismo nombre antes especificado bajo la extensión __.meta__. Puedes cargar este grafo al grafo default

In [ ]:
#Notar que comenzamos con un grafo vacío
tf.reset_default_graph()

#Lo siguiente carga la estructura del grafo
saver = tf.train.import_meta_graph("./tmp/my_model_final.ckpt.meta", clear_devices=True)
theta = tf.get_default_graph().get_tensor_by_name("theta:0")

with tf.Session() as sess:
    #Lo siguiente restaura el estado del grafo (i.e., los valores de las variables)
    saver.restore(sess, "./tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval()
    
print('Best theta restored:')
print(best_theta_restored)

#### Visualizando el Grafo Computacional ####

Ejecuta la siguiente celda para crear las funciones que graficarán el grafo computacional creado previamente

In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = b"<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

Notar que el último grafo que fue cargado fue el grafo restaurado desde el archivo .meta (antes de eso el grafo fue reiniciado)

<img

In [ ]:
show_graph(tf.get_default_graph())

### Modularidad

Supongamos que queremos crear un grafo que calcule dos ReLUs y sume sus outputs. Esto se puede realizar en TensorFlow como sigue

In [ ]:
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1")
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1")
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1")
relu2 = tf.maximum(z2, 0., name="relu2")

output = tf.add(relu1, relu2, name="output")

Este código funciona bien, pero es muy repetitivo. Una alternativa mejor es definir una función que realice esta tarea y pueda automáticamente generar todas las ReLUs que necesitemos. TensorFlow se asegura de que los nombres no se repitan, por lo que las ReLUs se irán denominando relu, relu_1, relu_2, ...

In [ ]:
tf.reset_default_graph()

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

In [ ]:
relus

O aún mejor, usando __name scope__ podemos definir los pesos y bias y el resto de las operaciones dentro de un _name scope_ llamado 'relu'. Esto permite otorgar un contexto a ciertas operaciones que definamos

In [ ]:
tf.reset_default_graph()

def relu(X):
    with tf.name_scope("relu"):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
        return tf.maximum(z, 0., name="max")
    
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

### Ejemplo 2: Construyendo una Red Neuronal en TensorFlow

En este ejercicio se construirá una __red neuronal feedforward__ con dos capas para modelar el dataset MNIST, el cual contiene imágenes de dígitos del 0 al 9 escritos a mano. Se mostrará que con un simple modelo la red neuronal es capaz de predecir a qué dígitos corresponden las imágenes nunca vistas antes por nuestra red con altos niveles de precisión!

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

In [ ]:
X_train = mnist.train.images
X_val = mnist.validation.images
X_test = mnist.test.images
y_train = mnist.train.labels.astype("int")
y_val = mnist.validation.labels.astype("int")
y_test = mnist.test.labels.astype("int")

¿Cómo se ven los datos descargados de MNIST?

In [ ]:
y_train[0:20]

In [ ]:
X_train[4]

<img src="https://github.com/nicoaramayo/imgsTFBasics/raw/master/mnist1.PNG" alt="Drawing" style="width: 500px;"/>

In [ ]:
print('Hay',X_train.shape[0],'datos de entrenamiento')
print('Hay ',X_val.shape[0],'datos de validación')
print('Hay',X_test.shape[0],'datos de testeo')
print('Hay  ',X_train.shape[1],'atributos por imagen')

<img src="https://github.com/nicoaramayo/imgsTFBasics/raw/master/mnist1.PNG" alt="Drawing" style="width: 500px;"/>

### Fase de construcción

In [ ]:
#Pixeles MNIST
n_inputs = 28*28
#Número de unidades ocultas en la primera capa
n_hidden1 = 300
#Número de unidades ocultas en la segunda capa
n_hidden2 = 100
#Número de clases (particular para MNIST)
n_outputs = 10

tf.reset_default_graph()

#Los parámetros de shape para X son None porque no sabemos apriori el tamaño de los batchs (y pueden ser variables),
#y n_inputs es fijo (cada atributo corresponde a un pixel)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

Ahora crearemos la red neuronal. El placeholder de X funcionará como la capa de input, y resta crear las dos capas escondidas más la capa de output. También se debe definir la función de costos y el algoritmo de optimización. Luego, definiremos los name scopes:<br>
> -__dnn__: red neuronal <br>
> -__loss__: función objetivo <br>
> -__train__: entrenamiento <br>
> -__eval__: obtiene métricas de desempeño

In [ ]:
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                              activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    
with tf.name_scope("loss"):
    #El argumento 'logits' son las log probabilidades no escaladas definidas en la parte anterior
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                              logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    #tf.nn.in_top_k me dice si mis targets están en las primeras k predicciones. En este caso las predicciones son
    #'logits', los targets son 'y', y k=1. Luego, retorna un tensor con booleans
    correct = tf.nn.in_top_k(logits, y, 1)
    #tf.cast pasa un tensor a otro tipo de dato, en este caso, tf.float32. Luego calculamos la media entre los 0s (errores)
    #y 1s (aciertos)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

El grafo construido es una red neuronal feedforward (o _fully connected_) como se muestra en la imagen

<img src="https://github.com/nicoaramayo/imgsTFBasics/raw/master/ffnn.png" alt="Drawing" style="width: 1000px;"/>

### Fase de ejecución

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epochs = 20
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            #Ya está implementado obtener los batches con .next_batch(batch_size)
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images,
                                            y: mnist.validation.labels})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images,
                                            y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy:", acc_val, "Test accuracy:", acc_test)

    save_path = saver.save(sess, "./my_model_final.ckpt")

Luego del entrenamiento, podemos ir revisando qué es lo que predice nuestra red neuronal en el set de testeo restaurando el modelo

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_final.ckpt")
    X_new_scaled = mnist.test.images
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

Visualicemos ahora los resultados en una matriz de confusión

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    
    if normalize:
        cm = 100 * cm / cm.sum(1)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.0f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
#Calcular la matriz de confusión
cnf_matrix = confusion_matrix(mnist.test.labels, y_pred)
np.set_printoptions(precision=2)
class_names = range(10)

#Plotear la matriz de confusión no-normalizada
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

#Plotear la matriz de confusión normalizada
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

Con este tutorial ya manejas los conceptos más importantes para poder desarrollar tus propios modelos TensorFlow, incluyendo poder implementar una red neuronal en el clásico dataset MNIST! <br>
Los conceptos que ya manejas son:
> + Crear __grafos computacionales__ <br>
> + Manejar múltiples grafos computacionales <br>
> + Dentro de los grafos: <br>
>> + Crear __variables__ y operaciones <br>
>> + Iniciar __sesiones__ <br>
>> + Y ejecutarlas <br>
> + Implementar __descenso del gradiente__ y __descenso del gradiente por minibatches__ (manejando nodos __placeholders__) <br>
> + __Guardar__ y __restaurar__ modelos <br>
> + Organizar tu fase de construcción en secciones bajo __name scopes__ <br>
> + Implementar el grafo computacional para una __red neuronal__, entrenarla, y calcular las métricas de ajuste para evaluarla <br>